<a href="https://colab.research.google.com/github/nejohnson2/daily-ai/blob/main/notebooks/Day1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>